In [1]:
"""
Purpose:
Practice how to learn do practical applications
with the neuron processing


"""

'\nPurpose:\nPractice how to learn do practical applications\nwith the neuron processing\n\n\n'

# Importing Relevant Packages

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
import soma_extraction_utils as sm
import neuron
import neuron_visualizations as nviz
from pathlib import Path

# ----------------- Application 1: Inhibitory vs Exctiatory ---------------

In [ ]:
Path("../test_neurons/spine_detection/excitatory_meshafterparty/")


In [19]:
from pathlib import Path
def strip_neuron_filename(neuron_path):
    neuron_path = Path(neuron_path)
    return "_".join(neuron_path.name.split("_")[:3])

# Excitatory Data

In [27]:
# getting all of the excitatory preprocessed neurons


cell_type = "excitatory"

curr_dir = Path(f"../test_neurons/spine_detection/{cell_type}_meshafterparty/")
neuron_files = list(curr_dir.iterdir())
excitatory_process_cell_files = [k for k in neuron_files if f".pbz2" in str(k)]
excitatory_process_cell_files

[PosixPath('../test_neurons/spine_detection/excitatory_meshafterparty/89309550087617165_excitatory_5_meshafterparty.pbz2'),
 PosixPath('../test_neurons/spine_detection/excitatory_meshafterparty/98887946639410170_excitatory_6_meshafterparty.pbz2'),
 PosixPath('../test_neurons/spine_detection/excitatory_meshafterparty/95442489112204099_excitatory_7_meshafterparty.pbz2'),
 PosixPath('../test_neurons/spine_detection/excitatory_meshafterparty/92700513338137592_excitatory_3_meshafterparty.pbz2'),
 PosixPath('../test_neurons/spine_detection/excitatory_meshafterparty/97626050552733538_excitatory_9_meshafterparty.pbz2'),
 PosixPath('../test_neurons/spine_detection/excitatory_meshafterparty/100917645171610430_excitatory_8_meshafterparty.pbz2')]

In [28]:
excitatory_mesh_files = [Path("../test_neurons/spine_detection/") / Path(strip_neuron_filename(k)) 
                         for k in excitatory_process_cell_files]
excitatory_mesh_files

[PosixPath('../test_neurons/spine_detection/89309550087617165_excitatory_5'),
 PosixPath('../test_neurons/spine_detection/98887946639410170_excitatory_6'),
 PosixPath('../test_neurons/spine_detection/95442489112204099_excitatory_7'),
 PosixPath('../test_neurons/spine_detection/92700513338137592_excitatory_3'),
 PosixPath('../test_neurons/spine_detection/97626050552733538_excitatory_9'),
 PosixPath('../test_neurons/spine_detection/100917645171610430_excitatory_8')]

In [47]:
import system_utils as su
for e_file in excitatory_mesh_files:
    print(su.get_file_size(str(e_file) + ".off"))

90083909
36159681
15498361
15094439
14430210
66503149


# Inhibitory Data

In [29]:
# getting all of the excitatory preprocessed neurons


cell_type = "inhibitory"

curr_dir = Path(f"../test_neurons/spine_detection/{cell_type}_meshafterparty/")
neuron_files = list(curr_dir.iterdir())
inhibitory_process_cell_files = [k for k in neuron_files if f".pbz2" in str(k)]
inhibitory_process_cell_files

[PosixPath('../test_neurons/spine_detection/inhibitory_meshafterparty/100224677766544793_inhibitory_2_meshafterparty.pbz2'),
 PosixPath('../test_neurons/spine_detection/inhibitory_meshafterparty/90725377802114822_inhibitory_7_meshafterparty.pbz2'),
 PosixPath('../test_neurons/spine_detection/inhibitory_meshafterparty/91076260457105133_inhibitory_1_meshafterparty.pbz2')]

In [31]:
inhibitory_mesh_files = [Path("../test_neurons/spine_detection/") / Path(strip_neuron_filename(k)) 
                         for k in inhibitory_process_cell_files]
inhibitory_mesh_files

[PosixPath('../test_neurons/spine_detection/100224677766544793_inhibitory_2'),
 PosixPath('../test_neurons/spine_detection/90725377802114822_inhibitory_7'),
 PosixPath('../test_neurons/spine_detection/91076260457105133_inhibitory_1')]

# Computing Spine Density

In [39]:
import skeleton_utils as sk
import neuron_utils as nru
def compute_spine_density(current_neuron,
                         verbose = True):
    """
    Purpose: To compute the number of spines per skeleton segment length
    for a certain decomposed cell (for each soma)
    
    Application: Discriminating between inhibitory and excitatory cells
    
    Pseudocode:
    For each soma in the neuron:
        0) Get all of the limbs that are touching the soma
        1) Get total number of spines on all touching limbs
        2) Get all skeletal length of touching limbs
        3) Calculate the spine density
    
    Return a dictionary mapping soma id to n_spines/segment_length
    """

    soma_idx_list = current_neuron.get_soma_node_names(int_label=True)
    soma_to_spine_density = dict([(k,0) for k in soma_idx_list])
    #For each soma in the neuron:
    for sm_idx in soma_to_spine_density:
        #0) Get all of the limbs that are touching the soma
        curr_touching_limbs = current_neuron.get_limbs_touching_soma(sm_idx)
        curr_total_spines = 0
        curr_total_skeletal_length = 0
        for l_idx in curr_touching_limbs:
            curr_limb = current_neuron[l_idx]
            #1) Get total number of spines on all touching limbs
            curr_total_spines += len(curr_limb.spines)
            
            #2) Get all skeletal length of touching limbs
            curr_total_skeletal_length += sk.calculate_skeleton_distance(curr_limb.skeleton)
        
        #3) Calculate the spine density
        spine_density = curr_total_spines/curr_total_skeletal_length
        if verbose:
            print(f"Soma_idx {sm_idx}: n_spines = {curr_total_spines},"
                  f" skeletal_length = {curr_total_skeletal_length}, spine_density={spine_density}")
        soma_to_spine_density[sm_idx] = spine_density
        
    #Return a dictionary mapping soma id to n_spines/segment_length
    return soma_to_spine_density

# Getting Excitatory Spine Density

In [40]:
excitatory_spine_density = []
for decomp_file,mesh_file in zip(excitatory_process_cell_files,excitatory_mesh_files):
    print(f"\n\n----- Working on {decomp_file} -------")
    n = nru.decompress_neuron(decomp_file,
                                           original_mesh=mesh_file)
    spine_density_mapping = compute_spine_density(current_neuron=n,
                         verbose = True)
    
    excitatory_spine_density.append(spine_density_mapping)



----- Working on ../test_neurons/spine_detection/excitatory_meshafterparty/89309550087617165_excitatory_5_meshafterparty.pbz2 -------
Decompressing Neuron in minimal output mode...please wait
Soma_idx 0: n_spines = 2566, skeletal_length = 6601029.838396014, spine_density=0.0003887272232999803
Soma_idx 1: n_spines = 3724, skeletal_length = 9397451.851198383, spine_density=0.00039627763557257363
Soma_idx 2: n_spines = 3350, skeletal_length = 7993082.478893886, spine_density=0.0004191124023611459
Soma_idx 3: n_spines = 3244, skeletal_length = 8246551.236058918, spine_density=0.00039337656520161627


----- Working on ../test_neurons/spine_detection/excitatory_meshafterparty/98887946639410170_excitatory_6_meshafterparty.pbz2 -------
Decompressing Neuron in minimal output mode...please wait
Soma_idx 0: n_spines = 883, skeletal_length = 5475102.147731724, spine_density=0.00016127552987587956


----- Working on ../test_neurons/spine_detection/excitatory_meshafterparty/95442489112204099_excit

In [41]:
excitatory_spine_density

[{0: 0.0003887272232999803,
  1: 0.00039627763557257363,
  2: 0.0004191124023611459,
  3: 0.00039337656520161627},
 {0: 0.00016127552987587956},
 {0: 0.0004489622874258387},
 {0: 0.00027403412108793963},
 {0: 0.000473994471738071},
 {0: 0.0004980678100603454, 1: 0.00030894569289822245}]

# Getting Inhibitory Spine Density

In [42]:
inhibitory_spine_density = []
for decomp_file,mesh_file in zip(inhibitory_process_cell_files,inhibitory_mesh_files):
    print(f"\n\n----- Working on {decomp_file} -------")
    n = nru.decompress_neuron(decomp_file,
                                           original_mesh=mesh_file)
    spine_density_mapping = compute_spine_density(current_neuron=n,
                         verbose = True)
    
    inhibitory_spine_density.append(spine_density_mapping)



----- Working on ../test_neurons/spine_detection/inhibitory_meshafterparty/100224677766544793_inhibitory_2_meshafterparty.pbz2 -------
Decompressing Neuron in minimal output mode...please wait
Soma_idx 0: n_spines = 392, skeletal_length = 3945440.7023955313, spine_density=9.935518730822428e-05


----- Working on ../test_neurons/spine_detection/inhibitory_meshafterparty/90725377802114822_inhibitory_7_meshafterparty.pbz2 -------
Decompressing Neuron in minimal output mode...please wait
Soma_idx 0: n_spines = 19, skeletal_length = 652430.3469801011, spine_density=2.9121882646852865e-05


----- Working on ../test_neurons/spine_detection/inhibitory_meshafterparty/91076260457105133_inhibitory_1_meshafterparty.pbz2 -------
Decompressing Neuron in minimal output mode...please wait
Soma_idx 0: n_spines = 622, skeletal_length = 3422850.339899698, spine_density=0.00018171989372407877


In [49]:
nviz.visualize_neuron(n,
                     limb_branch_dict="all",
                     axis_box_off=False)


 Working on visualization type: mesh


In [48]:
nviz.plot_spines(n,
                )


 Working on visualization type: mesh
